### Basic stats and probability reminder

In [2]:
from IPython.display import Math, display


Let x be a random variable with a finite nuber of finite outcomes x1, x2 ... xk occring with probabilities p1,p2,...pk. The E(x) expected value of x is defined as:

In [11]:
display(Math('E(x)=\sum x_{i} p_{i}'))

<IPython.core.display.Math object>

The outcomes can be {1,2,3,4,5,6}The E(x) expected value of x is: